In [2]:
#!/usr/bin/python

'''
'ISEG API testing.ipynb' is exactly what it says on the box: a testing ground for the ISEG API control.
It may be useful to see how I figured out how to do things, but there is a lot of legacy code in here.
One should use 'ISEG API.ipynb' to actaully do anything useful

A. J. McCulloch, December 2018
'''

# Import modules
import requests
import pandas as pd
import numpy as np
import os
import sys
import time
import json
import websocket
from enum import Enum #Used to list all constants of a class 
from datetime import datetime, timezone, timedelta

In [3]:
# Define functions unrelated to control system (for programming)

# Remove the element "key" from a dictionary "dic" without mutating the old dictionary
def removekey(dic, key):
    # Copy the input dictionary dic
    r = dict(dic)
    # Delete the key for the copied dictionary
    del r[key]
    # Return the dictionary with key removed
    return r

#Flatten sublists into a single list
def flattenlist(x):
    flat_list = [item for sublist in x for item in sublist]
    return flat_list

#Return a list with entries [d1,d2,range(d3)].
#Useful for generating addresses for HV supplies
def addressreturn(d1,d2,d3):
    addr = [[d1,d2,z] for z in range(d3)]
    return addr

#Convert UNIX timestamps to a readable (YMD HMS) format 
def timestampconvert(unixts,toffset=0):    
    #Convert the time stamp from UNIX to datetime object
    act_time = datetime.fromtimestamp(unixts, timezone.utc)
    #If the offset is defined, offset the time stamp
    if toffset != 0:
        act_time += toffset
    #Convert time to UTC time    
    #utc_time = act_time.replace(tzinfo=pytz.utc)
    #Shift UTC time to local time
    local_time = act_time.astimezone()
    #print(local_time.strftime("%Y-%m-%d %H:%M:%S.%f%z (%Z)"))
    return local_time.strftime('%Y-%m-%d %H:%M:%S (%Z)')

In [4]:
#Define the channels for the ISEG HV supply.
#Format is module, address, number of channels at that address
#module numbers are [0,1,8] (no idea why last module is 8)
chaddresses = [[0,0,8],[0,1,4],[0,2,4],[0,3,4],[1,0,4],[8,0,4]]

#Voltage limits for channels
limits = flattenlist([[1000] * 4, [-1000] * 4,[20000] * 2,[-20000] * 2,[10000] * 4,[-10000] * 4,\
                      [30000] * 2,[-30000] * 2,[30000] * 2,[-30000] * 2])

#Channel IDs
chids = ('+1kV1','+1kV2','+1kV3','+1kV4','-1kV1','-1kV2','-1kV3','-1kV4',\
        '+20kV1','+20kV2','-20kV1','-20kV2','+10kV1','+10kV2','+10kV3',\
        '+10kV4','-10kV1','-10kV2','-10kV3','-10kV4','+30kV1','+30kV2',\
        '-30kV1','-30kV2','+30kV3','+30kV4','-30kV3','-30kV4')

#Generate a list of addresses for each channel
chret = [addressreturn(*x) for x in chaddresses]

#Fix the channels in modules 1 and 2 (even only)
for i in list(range(-2,0)): 
    chret[i] = [[x[0],x[1],2*x[2]] for x in chret[i]]

#Flatten the list of channels
chlist = flattenlist(chret)

""" 
The iCS2 internal clock is incorrect, and at time of writing, an internal time server was unavailable.
In more detail, I can't be fucked to make the atomchumps router an NTP, but I found a good resource for
doing do; see https://github.com/kvic-z/goodies-asuswrt/wiki/Install-NTP-Daemon-for-Asuswrt-Merlin
"""
def timefix():
    #Run simple request to get timestamp
    tiCS = float(requests.get(apiget+sessionid+'/0/0/*/Status.isAlive').json()[0]['c'][0]['d']['t'])
    #Establish difference between timestamp CFIB computer and iCS2
    tdiff = datetime.now(timezone.utc) - datetime.fromtimestamp(tiCS,timezone.utc)
    return tdiff

In [5]:
#Define classes
#Trigger warning for advanced Python users

#A class to house the iCS tasks.
class iCStasks:
    
    #Status (meausrement)
    VMEAS = 'Status.voltageMeasure'
    ALIVE = 'Status.isAlive'
    TMEAS = 'Status.temperature'
    #Control (set values)
    POWER = 'Control.on'
    VSET = 'Control.voltageSet'

    #Can be updated dynamically but not overwritten
    #Adapted from http://www.siafoo.net/snippet/108
    def __setattr__(self, attr, value):
        if hasattr(self, attr):
            raise ValueError('Attribute %s already has a value and so cannot be written to' % attr)

        self.__dict__[attr] = value

#Class to store channel information        
class iCSchannels:
    
    #define a dictionary of limits
    limit = dict(zip(chids,limits))
    #define a dictionary of channel locations
    location = dict(zip(chids,chlist))  
     
#Used to define a single electrode voltage and execute the adjustment
class voltsetsinge:
    def __init__(self, electrode, value):
        self.electrode = electrode
        self.value = value

    def setvoltage(self):
        #Fix this to make it actually set the voltage, not just print something!
        print("set the volage of electorde {} to {}".format(self.electrode, self.value))  

In [6]:
#Define functions for communicating with ISEG iCS

#ip address of the iCS2
#NOTE: this is on the internal atomchumps network; in future we plan to transition to the university network
ip = '192.168.68.237'
#Websocket port
wsport = '8080'
#username for iCS2
usr = 'admin'
#password for the iCS2
#NOTE: I have put in zero effort to make this secure and hence do not distribute
passwd = 'molly7802'

#Shorthand string for '/api/getItem/'
apiget = 'http://'+ip+'/api/getItem/'
#Shorthand string for '/api/setItem/'
apiset = 'http://'+ip+'/api/setItem/'

#Returns API Key to be identified for session
r = requests.get('http://'+ip+'/api/login/'+usr+'/'+passwd)
sessionid = r.json()['i']

#Get the time difference between this computer and the iCS2
tfix = timefix()

'''
Functions here are from the testing era and are poorly coded or outdated.
They are useful for reference, but will eventually be removed and replaced
'''

#returns a list of ['channel number', measured voltage, 'unit'] for address x (x <= 3)
def returnvolts(x):
    #Returns voltage value, unit and timestamp of channel 0 of module with address x of line y
    #requests.get(apiget+sessionid+'/0/x/y/Status.voltageMeasure')
    #Currently x <= 3, y = [7,3,3,3]
    voltages = requests.get(apiget+sessionid+'/0/'+str(x)+'/*/Status.voltageMeasure')

    chans = [voltages.json()[0]['c'][i]['d']['p']['c'] for i in range(len(voltages.json()[0]['c']))]
    volts = [voltages.json()[0]['c'][i]['d']['v'] for i in range(len(voltages.json()[0]['c']))]
    units = [voltages.json()[0]['c'][i]['d']['u'] for i in range(len(voltages.json()[0]['c']))]

    comb = [[chans[i],float(volts[i]),units[i]] for i in range(len(chans))]
    
    return comb

#returns a list of ['address', chanel number', measured voltage, 'unit']
def returnallvolts():
    #Returns voltage value, unit and timestamp of channel 0 of module with address x of line y
    #requests.get(apiget+sessionid+'/0/x/y/Status.voltageMeasure')
    #Currently x <= 3, y = [7,3,3,3]
    voltages = requests.get(apiget+sessionid+'/0/*/*/Status.voltageMeasure')
    adds = [voltages.json()[0]['c'][i]['d']['p']['a'] for i in range(len(voltages.json()[0]['c']))]
    chans = [voltages.json()[0]['c'][i]['d']['p']['c'] for i in range(len(voltages.json()[0]['c']))]
    volts = [voltages.json()[0]['c'][i]['d']['v'] for i in range(len(voltages.json()[0]['c']))]
    units = [voltages.json()[0]['c'][i]['d']['u'] for i in range(len(voltages.json()[0]['c']))]

    comb = [[adds[i],chans[i],float(volts[i]),units[i]] for i in range(len(chans))]
    
    return comb

#Defintes the voltage values given an input of ['address','line',voltage,'unit'] and powers the line on
def setvolts(ts):
    requests.get(apiset+sessionid+'/0/'+ts[0]+'/'+ts[1]+'/Control.voltageSet/'+str(ts[2])+'/'+ts[3])
    #requests.get(apiset+sessionid+'/0/'+ts[0]+'/'+ts[1]+'/Control.on/1')

#Sets the voltages for the coincidence experiment
def setcoincidence():
    toset=[['2','3',1650,'V'],['3','0',-1250,'V'],['3','1',-750,'V'],['3','2',-1600,'V']]
    for i in range(len(toset)):
        setvolts(toset[i])
    print('Voltages set and units powered on')

#powers off all lines of all addresses    
def turnoff():
    requests.get(apiset+sessionid+'/0/*/*/Control.on/0')
    print('All units powered off')    

#test to see if all units are alive    
def testalive():
    ar=requests.get(apiget+sessionid+'/0/*/*/Status.isAlive')
    for x in ar.json()[0]['c']:
        ecount = 0
        if x['d']['v'] != '1':
            print('There is an .isAlive error in channel '+x['d']['p']['a'])
            ecount += 1
    return ecount

#check temperature, returns array with temps of all addresses
def checktemp():
    tempr = requests.get(apiget+sessionid+'/0/*/*/Status.temperature')
    temps = []
    for x in tempr.json()[0]['c']:
        temps.append(float(x['d']['v']))
    return temps    

'''
Functions here are from the post-testing era and are just poorly coded.
They are the main functions but hopefully will be replaced by someone in the future
'''

#Generate a request to interface with the iCS. Inputs are electode ID (str) and task (str)
def generaterequest(eid, task, voltage=0, unit='V'):
    #Verify channel ID is actual channel
    if eid in iCSchannels.location:
        #Verify the task is a registered task
        #NOTE: this is a tasty little number
        if task in [getattr(iCStasks,y) for y in [x for x in iCStasks.__dict__.keys() if not x.startswith('_')]]:
            
            #Voltage measure
            if task == 'Status.voltageMeasure':
                #Generate the appropriate address string, must be of the form /*/*/*/
                tloc = ''.join(['/'+str(x) for x in iCSchannels.location[eid]])+'/'
                #Generate the request
                req = apiget+sessionid+tloc+task
                return req
            
            #Voltage set
            elif task == 'Control.voltageSet':
                #Generate the appropriate address string, must be of the form /*/*/*/
                tloc = ''.join(['/'+str(x) for x in iCSchannels.location[eid]])+'/'
                #Verify voltage unit
                if unit in ('V', 'kV'):
                    #Scale kV voltages to V
                    if unit == 'kV':
                        voltage *= 1000
                    #Verify voltage is less than channel limit
                    if abs(voltage) <= abs(iCSchannels.limit[eid]) and np.sign(voltage) == np.sign(iCSchannels.limit[eid]) \
                    or np.sign(voltage) == 0:
                        #Generate voltage string
                        setval = ''.join('/'+str(voltage)+'/'+unit)
                        #Generate the request
                        req = apiset+sessionid+tloc+task+setval
                    else:
                        print('The set voltage is outside the channel limit; it must be less \
                        than {} V'.format(iCSchannels.limit[eid]))
                        req = None
                else:
                    print('Invalid unit for voltage; use either V or kV')
                    req = None
                return req
        
        #Inform if task is not registered task
        else:
            print('The specified task {} does not exist. Registered tasks are stored in the iCStasks class'.format(task))
    #Inform of incorrect IDs        
    else:
        print('Channel ID {} does not exist. The channel ID must be registered; for a \
        list of registered IDs, execute print(chids)'.format(id))

#Execute a request and return the response
def execturerequest(request):
    req = requests.get(request)
    return req

#Extract information from a response
def interpretresponse(response):
    #Catch content in response
    try:
        #Convert response to useful list by extracting content of response
        content = response.json()[0]['c']
        if len(content) == 0:
            print('No content is returned in response')
            #Return nothing
            ds = None
        elif len(content) == 1:
            #Return a dictionary with the embedded data
            ds = content[0]['d']
        else:
            #Return a list of dictionaries of the embedded data
            ds=[content[x]['d'] for x in range(len(content))]

        #In the case of of voltage measurement, print the voltage (timestamped)
        if ds['i'] == iCStasks.VMEAS:
            """
            This is not working, the offest has issues, resulting in incorrect times
            It is in the bin for the moment.
            
            #Check to see if the time offset has been calculated
            try:
                tfix
            except NameError:
                tfix = timefix()
            else:
                pass     

            #print('The returned voltage was {} V [{}]'.format(ds['v'],timestampconvert(float(ds['t']),tfix)))
            """

            print('The returned voltage was {} V [{}]'.format(ds['v'],timestampconvert(time.time())))
    #If not content in response, look for trigger (information given in docs)
    #http://192.168.68.237//doc/iCSservice/iCSapiWebsocket_Docu.html
    except KeyError:
        content = response.json()[0]['trigger']
        if content == 'false':
            print('Server can not handle request or wrong input information given')
        elif content == 'true':
            print('Server has (suscessfully) processed the request')
        elif content == 'denied':
            print('Server denies request because of access restrictions')
        elif 'noData':
            print('No data available to this request (on getUpdate command)')
        else:
            pass
        
        ds = None
    
    return ds
            
#Return a dataframe of all measured voltages
def measureall(task):
    #Verify the task is a registered task
    #NOTE: this is a tasty little number
    if task in [getattr(iCStasks,y) for y in [x for x in iCStasks.__dict__.keys() if not x.startswith('_')]]:
        #Use API to pull data, returns a response
        rquest = requests.get(apiget+sessionid+'/*/*/*/'+task)
        #Convert response to useful list by extracting content of response
        content = rquest.json()[0]['c']
        #Return a list of dictionaries of the embedded data
        ds=[content[x]['d'] for x in range(len(content))]
        #Put data in a dataframe
        df=pd.DataFrame(ds)
        return df
    else:
        print('The specified task {} does not exist. Registered tasks are stored in the iCStasks class'.format(task))

# Example usage

In [ ]:
#Measure all voltages
measureall(iCStasks.VMEAS)

#Generate the request for controlling with the iCS
generaterequest('+1kV3',iCStasks.VMEAS)

#Interperate a response from a generated, executed request
#EX1: Measure voltage
interpretresponse(execturerequest(generaterequest('+1kV3',iCStasks.VMEAS)))
#EX2: Set voltage
interpretresponse(execturerequest(generaterequest('+1kV1',iCStasks.VSET, 11)))

In [ ]:
#Testing for voltage ramping

'''
This was the first program written to test the iCS control
In other words, it works but nothing more
'''

def main():
    
    sleep_time = 10
    
    #Initialise the voltage
    #Get the current set value
    vsr = requests.get(apiget+sessionid+'/0/0/0/Control.voltageSet')
    vset = float(vsr.json()[0]['c'][0]['d']['v'])
    #If it is not zero, make it zero
    if vset != 0:
        requests.get(apiset+sessionid+'/0/0/0/Control.voltageSet/0/V')
    
    #minumum voltage
    v_min = 50
    #maximum voltage
    v_max = 100
    #number of steps 
    v_steps = 11
    voltages=np.linspace(v_min,v_max,v_steps)
    
    for v in voltages:
        requests.get(apiset+sessionid+'/0/0/0/Control.voltageSet/'+str(v)+'/V')
        time.sleep(sleep_time)

# Testing

In [273]:
testmeas = interpretresponse(execturerequest(generaterequest('+1kV3',iCStasks.VMEAS)))
#testset = interpretresponse(execturerequest(generaterequest('+1kV1',iCStasks.VSET,10)))

The returned voltage was -0.016 V [2019-01-22 10:57:05 (AUS Eastern Daylight Time)]


In [ ]:
#Apparently instructions exist and can be returned in an xml file

import xml.etree.ElementTree

xml = execturerequest('http://192.168.68.237/api/getItemsInfo')
e = xml.etree.ElementTree.parse(xml).getroot()

#Found it here: http://192.168.68.237/api/getItemsInfo

In [287]:
#Trying to get multiple requests on a single event

r1, r2 = generaterequest('+1kV1',iCStasks.VMEAS), generaterequest('+1kV2',iCStasks.VMEAS)

In [309]:
raa = execturerequest('http://192.168.68.237/api/getItem/55004ed5d0f14-27/0/0/0/Status.voltageMeasure')
raa.json()

In [ ]:
"""
For a large number of requests, it seems that using asynchronous requests is a good idea:
http://skipperkongen.dk/2016/09/09/easy-parallel-http-requests-with-python-and-asyncio/

Also some work on creating a session rather than a butt-tonne of requests
https://medium.com/@anthonypjshaw/python-requests-deep-dive-a0a5c5c1e093
"""

In [297]:
r12 = execturerequest(r1)
r12.json()

In [ ]:
e1 = voltset(1,1000)
e1.setvoltage()

In [338]:
e1 = iCS2(chids[0],chlist[0],limits[0])

In [342]:
e1.address

[0, 0, 0]

In [388]:
#initialise electrodes
for x in elabels:
    #Use the index of the element x in elabels for the electrode properties
    i = elabels.index(x)
    #Set a string to a variable and then define class attributes
    vars()[x] = iCS2(chids[i],chlist[i],limits[i])

In [395]:
e1.address in chlist

True

In [412]:
{
	"i": "55005e1a49cad-46",    // session id, received by login
	"t": "request",             // type request
	"c": [                      // content array, collects different request pakets
		{                       // request paket object
			"c": "getItem",     // command: getItem
			"p": {
				"p": {          // path object
					"l": "0",   // line (0)
					"a": "*",   // address (every device in line 0)
					"c": "*"    // channel (every channel of every device in line 0)
				},
				"i": "VoltageMeasure",  // item type (measured Voltage)
				"v": "",        // must be empty
				"u": ""         // must be empty
			}
		}
	],
	"r": "websocket" // session response type: websocket => push mode communication | xhr => polling mode communication
}

SyntaxError: invalid syntax (<ipython-input-412-4d9cf5fe8c92>, line 2)

# Using the API for passing .json instructions

In [446]:
import json

vmeascom = {"i": sessionid,"t": "request","c": [{"c": "getItem","p": {"p": {"l": "0","a": "0", "c": "0"},"i": "VoltageMeasure","v": "","u": ""}}],
"r": "websocket"}

data_json = json.dumps(tt)

parameters = {"lat": 37.78, "lon": -122.41}
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

parameters

payload = {'json_payload': data_json, 'apikey': sessionid}

In [ ]:
[{"t":"response","c":[{"e":"itemUpdated","d":{"p":{"l":"0","a":"0","c":"0"},"i":"Status.voltageMeasure","v":"","u":"","t":"1529236198.6850"}}]}]

In [449]:
rzm = requests.get('http://192.168.68.237/', params=tt)
rzm.content

In [442]:
rzz = execturerequest(generaterequest('+1kV3',iCStasks.VMEAS))

In [ ]:
interpretresponse(execturerequest(generaterequest('+1kV1',iCStasks.VSET, 11)))

In [411]:
generaterequest('+1kV1',iCStasks.VSET, 11)

'http://192.168.68.237/api/setItem/55004ed5d0f14-27/0/0/0/Control.voltageSet/11/V'

In [422]:
sessionid

'55004ed5d0f14-27'

In [423]:
interpretresponse(execturerequest(generaterequest('+1kV3',iCStasks.VMEAS)))

The returned voltage was -0.009 V [2019-01-25 10:55:59 (AUS Eastern Daylight Time)]


{'i': 'Status.voltageMeasure',
 'p': {'a': '0', 'c': '2', 'l': '0'},
 't': '1529227833.3280',
 'u': 'V',
 'v': '-0.009'}

In [457]:
json.dumps(loginreq)

'{"i": "", "t": "login", "c": {"l": "admin", "p": "molly7802", "t": ""}, "r": "websocket"}'

In [421]:
rtest1 = requests.post('http://192.168.68.237/api/', json = json_data)
rtest1.status_code

404

# Using websocket for control

Instructions from the iCS2 regarding wc connection

- Open Websocket connection on iCS Host IP and Websocket port (default 8080, see Javascript example)
- Send login message
- Receive response message with Session ID information, if false a Trigger message with false will be send
- Send request with setItem/getItem/(getUpdate) commands and the session id
- Once a getItem or getUpdate command is sent in the session to a specific Object this info will be updated by push
- Define the session mode using the responsetype "r" between direct push communication (r: "websocket") or polling mode, where the client requests updates using the "getUpdate" Command (r: "xhr")
- Process incoming response or info messages
- Use heartbeat messages for alive tests
- Logout

In [117]:
import websocket
import json

#Login
ws = websocket.create_connection("ws://192.168.68.237:8080")
loginreq = {"i": "", "t": "login", "c": {"l": "admin", "p": "molly7802", "t": ""}, "r": "websocket"}
ws.send(json.dumps(loginreq))
sessionid = json.loads(ws.recv())['i']

In [91]:
#Logout
logoutreq = {"i": sessionid, "t": "logout", "c": {}, "r": "websocket"}
ws.send(json.dumps(logoutreq))
ws.close()

In [125]:
#Measure voltage. This worked after a long period of not working, so not sure what is going on...
wsvmeas = {"i": sessionid, "t": "request", "c": [{"c": "getItem", "p": {"p": {"l": "*", "a": "*", "c": "*"}, "i": "Status.voltageMeasure", "v": "", "u": ""}}]}
measreq = json.dumps(wsvmeas)
ws.send(measreq)
measout = json.loads(ws.recv())
measout

[{'c': [{'d': {'i': 'Status.voltageMeasure',
     'p': {'a': '0', 'c': '0', 'l': '0'},
     't': '1538413427.9929',
     'u': 'V',
     'v': '0.006'},
    'e': 'itemUpdated'},
   {'d': {'i': 'Status.voltageMeasure',
     'p': {'a': '0', 'c': '1', 'l': '0'},
     't': '1538413427.9955',
     'u': 'V',
     'v': '-0.001'},
    'e': 'itemUpdated'},
   {'d': {'i': 'Status.voltageMeasure',
     'p': {'a': '0', 'c': '2', 'l': '0'},
     't': '1538413427.9934',
     'u': 'V',
     'v': '-0.011'},
    'e': 'itemUpdated'},
   {'d': {'i': 'Status.voltageMeasure',
     'p': {'a': '0', 'c': '3', 'l': '0'},
     't': '1538413427.9938',
     'u': 'V',
     'v': '-0.016'},
    'e': 'itemUpdated'},
   {'d': {'i': 'Status.voltageMeasure',
     'p': {'a': '0', 'c': '4', 'l': '0'},
     't': '1538413427.9942',
     'u': 'V',
     'v': '-0.047'},
    'e': 'itemUpdated'},
   {'d': {'i': 'Status.voltageMeasure',
     'p': {'a': '0', 'c': '5', 'l': '0'},
     't': '1538413427.9947',
     'u': 'V',
     'v': 

In [138]:
turnon = {"i": sessionid, "t": "request", "c": [{"c": "setItem", "p": {"p": {"l": "0", "a": "0", "c": "0"}, "i": "Control.on", "v": "1", "u": ""}}]}
onreq = json.dumps(turnon)
ws.send(onreq)

156

In [136]:
content = measout[0]['c']
[content[x]['d'] for x in range(len(content))]

[{'i': 'Status.voltageMeasure',
  'p': {'a': '0', 'c': '0', 'l': '0'},
  't': '1538413427.9929',
  'u': 'V',
  'v': '0.006'},
 {'i': 'Status.voltageMeasure',
  'p': {'a': '0', 'c': '1', 'l': '0'},
  't': '1538413427.9955',
  'u': 'V',
  'v': '-0.001'},
 {'i': 'Status.voltageMeasure',
  'p': {'a': '0', 'c': '2', 'l': '0'},
  't': '1538413427.9934',
  'u': 'V',
  'v': '-0.011'},
 {'i': 'Status.voltageMeasure',
  'p': {'a': '0', 'c': '3', 'l': '0'},
  't': '1538413427.9938',
  'u': 'V',
  'v': '-0.016'},
 {'i': 'Status.voltageMeasure',
  'p': {'a': '0', 'c': '4', 'l': '0'},
  't': '1538413427.9942',
  'u': 'V',
  'v': '-0.047'},
 {'i': 'Status.voltageMeasure',
  'p': {'a': '0', 'c': '5', 'l': '0'},
  't': '1538413427.9947',
  'u': 'V',
  'v': '0.002'},
 {'i': 'Status.voltageMeasure',
  'p': {'a': '0', 'c': '6', 'l': '0'},
  't': '1538413427.9951',
  'u': 'V',
  'v': '0.003'},
 {'i': 'Status.voltageMeasure',
  'p': {'a': '0', 'c': '7', 'l': '0'},
  't': '1538413427.9925',
  'u': 'V',
  'v'

In [123]:
wsvmeas = wstaskgen([0,0,1], wstask = iCStasks.VMEAS)
measreq = json.dumps(wsvmeas)
ws.send(measreq)
measout = json.loads(ws.recv())
measout

[{'c': [{'d': {'i': 'Status.voltageMeasure',
     'p': {'a': '0', 'c': '1', 'l': '0'},
     't': '1538413392.1755',
     'u': 'V',
     'v': '-0.002'},
    'e': 'itemUpdated'}],
  't': 'response'}]

In [124]:
wstaskgen([0,0,1], wstask = iCStasks.VMEAS)

{'c': [{'c': 'getItem',
   'p': {'i': 'Status.voltageMeasure',
    'p': {'a': '0', 'c': '1', 'l': '0'},
    'u': '',
    'v': ''}}],
 'i': '55004ed5d0f14-18',
 'r': 'websocket',
 't': 'request'}

In [118]:
ws.close()

In [ ]:
"""
Polling is finally working. Need to get this functioning on the control software proper
Then implement multiple tasks
"""

In [16]:
ws.connected

True

In [173]:
statpower = {"i": sid,"t": "request","c": [{"c": "getItem","p": {"p": {"l": "","a": "", "c": ""},"i": "Status.power","v": "","u": ""}}]}
pwrreq = json.dumps(statpower)
ws.send(pwrreq)
pwrout = json.loads(ws.recv())


#setout = json.loads(ws.recv())

In [11]:
#Function to generate the websocket tasks. Mainly geared for setting electrode values
#Inputs are electrode address (e.address), the set voltage, the task to complete and the structure type (usally request but can be login/logout)
def wstaskgen(eloc = [None, None, None], voltage = 0, units = 'V', wstask = iCStasks.VSET, stuct = 'request'):
    #Session mode ("websocket" or "xhr"), see iCS2 docs
    r = 'websocket'
    #Structure type
    t1 = stuct
    
    #Generate login task parameters, must be {"i": "", "t": "login", "c": {"l": "user", "p": "password", "t": ""}, "r": "websocket"} 
    if t1 == 'login':
        #Session ID
        i = ''
        #Username
        l = usr
        #Password
        try:
            p = passwd
        except KeyError:
            p = getpass.getpass("Password for iCS2 module:")        
        t2 = ''
        c1 = {'l': l, 'p': p, 't': t2}    
    
    #Generate logout task parameters, must be {"i": session ID, "t": "logout", "c": {}, "r": "websocket"}
    elif t1 == 'logout':
        i = sessionid
        c1 = {}
    
    #Generate requst task
    elif t1 == 'request':
        #Session ID
        i = sessionid
        #Generate task for setting the voltage, must be {"i": session ID, "t": "request", "c": [{"c": "setItem","p": {"p": {"l": "0","a": "2", "c": "1"},"i": "Control.voltageSet","v": "0","u": "V"}}], "r": "websocket"}
        if wstask == iCStasks.VSET:
            c2 = 'setItem'
            [line, add, chan] = [str(i) for i in eloc]
            item = wstask
            v = voltage
            u = units
            p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': wstask, 'v': v, 'u': u}
            
            c1 = [{'c': c2, 'p': p}]
        #Generate task for measuring the voltage, must be {"i": sessionid, "t": "request", "c": [{"c": "getItem", "p": {"p": {"l": "0", "a": "0", "c": "7"}, "i": "Status.voltageMeasure", "v": "", "u": ""}}]}
        elif wstask == iCStasks.VMEAS:
            c2 = 'getItem'
            [line, add, chan] = [str(i) for i in eloc]
            item = wstask
            v = ''
            u = ''
            p = {'p': {'l': line, 'a': add, 'c': chan}, 'i': wstask, 'v': v, 'u': u}
            
            c1 = [{'c': c2, 'p': p}]
            
    #The generated task (see iCS2 docs for unclear discussion of format)
    gentask = {'i': i, 't': t1, 'c': c1, 'r': r}
    
    return gentask

#A function to execute websocket tasks (usually generated with "wstaskgen")
def executews(task, responselimit = 5):
    
    #The session ID is a global variable
    global sessionid
    
    #Verify websocket has been created
    try:
        ws
    except NameError:
        #If socket has not been created, create it
        ws = websocket.create_connection('ws://'+ip+':'+wsport)
        
    #If session ID exists, one can use the websocket connection
    try:
        sessionid
    #If session ID is not defined, use websocket to initiate session
    except NameError:
        #If executing a login command, don't establiash a connection
        if task['t'] == 'login':
            pass
        #Otherwise, establish a connection
        else:
            #Open websocket connection
            ws.send(json.dumps(wstaskgen(stuct = 'login')))
            #Get the session ID
            sessionid = json.loads(ws.recv())['i']
            #Set a flag to we don't make two connections
            loginflag = True
            print("No existing session found. Websocket communication established")
    
    #Prepare data packet
    packet = json.dumps(task)
    
    #For login task
    if task['t'] == 'login':
        if loginflag == True:
            #If the flag is true, login has occured
            pass
        else:
            #Send data packet
            ws.send(packet)
            sessionid = json.loads(ws.recv())['i']
            print("Websocket communication sucessfully established")
        response = None
    
    #For logout task
    elif task['t'] == 'logout':
        #Send data packet
        ws.send(packet)
        print("Websocket server logout completed")
        ws.close()
        print("Websocket closed")
        response = None
    
    #For request task
    elif task['t'] == 'request':
        #Setting a value 
        if task['c'][0]['c'] == 'setItem':
            #Send data packet
            ws.send(packet)
            response = None
        #Requesting a value
        elif task['c'][0]['c'] == 'getItem':
            #Send data packet
            ws.send(packet)
            #Receive response
            #response = json.loads(ws.recv())
            
            reclimit = 0
            while reclimit < responselimit:
                received = json.loads(ws.recv())            
                response = intrespws(received, task)
                if response == None:
                    reclimit += 1
                    if reclimit == responselimit - 1:
                        print('No response for iCS2, check connection')
                        reclimit = responselimit
                else:
                    try:                
                        received[0]['trigger']
                        reclimit += 1
                        if reclimit == responselimit - 1:
                            print('Invalid request, check channel address')
                            reclimit = responselimit
                    except (KeyError, TypeError):
                        reclimit = responselimit

    #Any interpretation that does not require a response shouldn't return anything
    if response == None:
        measdata = None
    else:
        pass
    
    return response

#Interperet websocket response for a given task (function called from executews)
def intrespws(response, task):
    #Interpret the response
    try:
        #Get the data packet from the respose
        recdatadict = response[0]['c'][0]['d']
    except (KeyError, TypeError):
        print("No data packet returned from iCS2 (check channel address)")
        recdatadict = None

    #Process the content of the data packet
    if recdatadict != None:
        #Extract the useful information from returned data
        try:
            datakeys = ['i','p','v','u','t']
            #Get the data
            recdata = [recdatadict.get(key) for key in datakeys]
        except KeyError:
            print("Unexpected data returned from iCS2")
            datakeys, recdata = None, None
        
        #Check the data is what was requested
        if recdata != None and recdata[:2] == [task['c'][0]['p'].get(key) for key in ['i','p']]:
            #Store the useful data
            measdata = recdata[2:]
        else:
            measdata = None
            print('Returned data different from requested information')
    else:
        measdata = None
    
    return measdata

In [ ]:
executews(wstaskgen([0,0,8], wstask = iCStasks.VMEAS))

In [ ]:
ws.close()

In [13]:
ws.connected

True

In [21]:
def testset(V):
    return wstaskgen([0,0,0], V)

### The example below demonstrates that the session ID for websocket and API is interchangable

In [17]:
#Create a session ID
try:
    sessionid
except NameError:
    #Create a session ID via the API, not via websocket
    r = requests.get('http://'+ip+'/api/login/'+usr+'/'+passwd)
    try:
        sessionid = r.json()['i']
        print("Session successfully created")
    except KeyError:
        sessionid = None
        print("Session not created. Check password or iCS2 connection")
else:
    pass

#Open the websocket
ws = websocket.create_connection("ws://192.168.68.237:8080")
#Test
ws.send(json.dumps(wstaskgen([0,0,0], 0)))

180

In [29]:
ws.send(json.dumps(wstaskgen(stuct = 'login')))
WSsessionid = json.loads(ws.recv())['i']

In [39]:
try:
    sessionid
#If session ID is not defined, use websocket to initiate session
except NameError:
    #Open websocket connection
    ws.send(json.dumps(wstaskgen(stuct = 'login')))
    sessionid = json.loads(ws.recv())['i']
    loginflag = True
    print("Websocket communication sucessfully established, 1")

In [42]:
type(sessionid)

str

In [ ]:
#Define functions for communicating with ISEG iCS
class iCS2(object):
    
    #Set attributes
    def __init__(self, chid, address, limit, setpoint = 0, toset = 0, active = False, measured = None):
        self.chid = chid #Channel ID
        ##self.address = address #Channel location [line, address, channel]
        self.address = address #Channel location {'l': line, 'a': address, 'c': channel}
        self.limit = limit #Channel limit
        self.setpoint = setpoint #Current channel setpoint
        self.toset = toset #Value to set when setting multiple electrodes
        self.active = active #Is the channel active
        self.powered = powered #Is the channel active
        self.measured = measured #Last measured voltage
        
    #Function for powering electrodes
    def power(self, turn_on = True):
        if turn_on == True:
            value = 1
        elif turn_on == False:
            value = 0
            
        exectureWSrequest(generateWSrequest(self, value, wstask = iCStasks.POWER))
        
    
    #Function for setting the voltage of a single electrode
    def setvoltage(self, voltage=0, units='V', useAPI = False):
        #Set the voltage via API
        if useAPI == True:
            interpretAPIresponse(exectureAPIrequest(generateAPIrequest(self, iCStasks.VSET, voltage, units)))
        #Set voltage over websocket
        else:
            exectureWSrequest(generateWSrequest(self, voltage))
        if self.setpoint == voltage:
            print("Voltage of {} successfully set to {} {}".format(self.chid, voltage, units))
        else:
            print("Voltage of {} not updated".format(self.chid))
        
    #Function for measuring the voltage of a single electrode
    def measurevoltage(self, useAPI = False):
        #Measure the voltage using the API (slow)
        if useAPI == True:
            vmeas = interpretAPIresponse(exectureAPIrequest(generateAPIrequest(self,iCStasks.VMEAS)))
        #Get the voltage via websocket
        else:
            vmeas = exectureWSrequest(generateWSrequest(self, wstask = iCStasks.VMEAS))
        
        self.measured = float(vmeas[0])
        if vmeas != None:
            print('The returned voltage was {} V [{}]'.format(vmeas[0],timestampconvert(time.time())))
        else:
            print('No voltage was returned')


In [139]:
testobj = {'c': [{'c': 'getItem',
   'p': {'i': 'Status.runningState',
    'p': {'a': '*', 'c': '*', 'l': '*'},
    'u': '',
    'v': ''}}],
 'i': '55004ed5d0f14-9',
 'r': 'websocket',
 't': 'request'}

In [140]:
testobj['t']

'request'

In [142]:
testobj['c'][0]['c']

'getItem'

In [144]:
ws.connected

True

In [ ]:
interpretWSresponse(response, task)